In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime

import pandas as pd
import numpy as np
from ta.volatility import BollingerBands
from ta.trend import MACD
from ta.momentum import RSIIndicator

In [27]:
# some =pd.read_csv('XAUUSD.csv')
some

,time,open,high,low,close,tick_volume,MA_daily,MA_weekly,RSI,BB_High,BB_Low,MACD,signal,RSI_signal,BB_signal,MACD_signal,MA_signal,Engulfing
0,2017-01-31 11:00:00,1198.45,1204.32,1197.53,1204.23,26378,1192.6794,1192.27010,65.694013,1203.664980,1192.537020,2.179186,1,0,1,1,1,0
1,2017-01-31 12:00:00,1204.23,1206.82,1203.68,1204.89,38616,1193.1338,1192.57645,66.653721,1204.512757,1192.861243,2.460562,1,0,1,1,1,0
2,2017-01-31 13:00:00,1204.88,1205.65,1203.27,1204.11,37947,1193.5702,1192.88415,64.362141,1205.260365,1192.700635,2.590749,1,0,0,1,1,0
3,2017-01-31 14:00:00,1204.08,1214.34,1203.94,1212.94,70737,1194.1904,1193.23945,74.887745,1208.366068,1191.385932,3.367612,1,1,1,1,1,1
4,2017-01-31 15:00:00,1212.94,1215.41,1211.77,1212.51,75073,1194.7566,1193.59475,73.745489,1210.654529,1190.619471,3.903585,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37575,2023-06-16 09:00:00,1963.15,1964.71,1963.15,1964.33,4098,1949.2434,1955.61310,66.591302,1963.886858,1952.480142,4.246097,1,0,1,1,1,0
37576,2023-06-16 10:00:00,1964.31,1965.61,1963.06,1964.85,3714,1949.6150,1955.63565,67.177643,1964.632911,1952.938089,4.388206,1,0,1,1,1,0
37577,2023-06-16 11:00:00,1964.91,1966.54,1963.91,1965.62,5336,1949.9344,1955.65960,68.071247,1965.373068,1953.447932,4.510962,1,0,1,1,1,0
37578,2023-06-16 12:00:00,1965.66,1967.93,1962.40,1962.61,7713,1950.1894,1955.66720,61.071618,1965.680284,1953.745716,4.315617,1,0,0,1,1,-1


In [2]:
# df.to_csv('btcusd.csv', index=False)

In [31]:
def apply_indicators(df):
    
    # Calculate the moving averages
    df['MA_daily'] = df['close'].rolling(window=50).mean()
    df['MA_weekly'] = df['close'].rolling(window=200).mean()

    # Calculate the Relative Strength Index (RSI)
    rsi_indicator = RSIIndicator(df['close'])
    df['RSI'] = rsi_indicator.rsi()

    # Calculate Bollinger Bands
    bollinger = BollingerBands(df['close'])
    df['BB_High'] = bollinger.bollinger_hband()
    df['BB_Low'] = bollinger.bollinger_lband()

    # Calculate MACD
    macd_indicator = MACD(df['close'])
    df['MACD'] = macd_indicator.macd()

    
    return data

In [32]:
def indicator_signal(df):
    # RSI rules
    df['RSI_signal'] = 0
    df.loc[df['RSI'] > 70, 'RSI_signal'] = 1
    df.loc[df['RSI'] < 30, 'RSI_signal'] = -1
    # Bollinger Bands rules
    df['BB_signal'] = 0
    df.loc[df['close'] > df['BB_High'], 'BB_signal'] = 1
    df.loc[df['close'] < df['BB_Low'], 'BB_signal'] = -1
    # MACD rules
    df['MACD_signal'] = 0
    df.loc[df['MACD'] > 0, 'MACD_signal'] = 1
    df.loc[df['MACD'] < 0, 'MACD_signal'] = -1
    # Moving Averages rules
    df['MA_signal'] = 0
    df.loc[df['close'] > df['MA_daily'], 'MA_signal'] = 1
    df.loc[df['close'] < df['MA_daily'], 'MA_signal'] = -1
    
    #engulfing signal 
    df['Engulfing'] = 0
    for i in range(1, len(df)):
        # Bullish engulfing condition
        if df.loc[i-1, 'open'] > df.loc[i-1, 'close'] and df.loc[i, 'open'] < df.loc[i, 'close'] and df.loc[i-1, 'open'] < df.loc[i, 'close'] and df.loc[i-1, 'close'] > df.loc[i, 'open']:
            df.loc[i, 'Engulfing'] = 1

        # Bearish engulfing condition
        elif df.loc[i-1, 'open'] < df.loc[i-1, 'close'] and df.loc[i, 'open'] > df.loc[i, 'close'] and df.loc[i-1, 'open'] > df.loc[i, 'close'] and df.loc[i-1, 'close'] < df.loc[i, 'open']:
            df.loc[i, 'Engulfing'] = -1
    
    # Remove nas
#     data = df.dropna()
    return data

In [33]:
data.columns

Index(['time', 'open', 'high', 'low', 'close', 'tick_volume', 'MA_daily',
       'MA_weekly', 'RSI', 'BB_High', 'BB_Low', 'MACD', 'signal', 'RSI_signal',
       'BB_signal', 'MACD_signal', 'MA_signal', 'Engulfing'],
      dtype='object')

# Getting data from the metatrader server

In [38]:
def get_data(symbol):

    # Getting data on the 15 minutes timeframe
    if not mt5.initialize():
        print("initialize() failed, error code =",mt5.last_error())
        quit()

    # set the symbol and timefram
    timeframe = mt5.TIMEFRAME_H1   # for one-minute bars

    #dates for retrieving the history
    date_from = datetime(2016, 12, 15)
    today = datetime.today()

    # get the history
    history = mt5.copy_rates_range(symbol, timeframe, date_from, today)

    if history is not None and len(history) > 0:
        # create DataFrame out of the obtained data
        rates_frame = pd.DataFrame(history).drop(['spread','real_volume'], axis =1)

        # convert time in seconds into the datetime format
        rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
    else:
        print("No data for the requested period")

    # terminate the connection to the MetaTrader 5 terminal
    mt5.shutdown()
    
    #adding indicators to retrieved data
    data_with_indicators = apply_indicators(rates_frame)
    data_with_indicators = data_with_indicators.reset_index(drop=True)
    data = indicator_signal(data_with_indicators)
    data.to_csv(f'{symbol}.csv', index=False)

    return data

In [39]:
# some =get_data('BTCUSD')
data =get_data('XAUUSD')
data

,time,open,high,low,close,tick_volume,MA_daily,MA_weekly,RSI,BB_High,BB_Low,MACD,signal,RSI_signal,BB_signal,MACD_signal,MA_signal,Engulfing
199,2017-01-31 11:00:00,1198.45,1204.32,1197.53,1204.23,26378,1192.6794,1192.27010,65.694013,1203.664980,1192.537020,2.179186,1,0,1,1,1,0
200,2017-01-31 12:00:00,1204.23,1206.82,1203.68,1204.89,38616,1193.1338,1192.57645,66.653721,1204.512757,1192.861243,2.460562,1,0,1,1,1,0
201,2017-01-31 13:00:00,1204.88,1205.65,1203.27,1204.11,37947,1193.5702,1192.88415,64.362141,1205.260365,1192.700635,2.590749,1,0,0,1,1,0
202,2017-01-31 14:00:00,1204.08,1214.34,1203.94,1212.94,70737,1194.1904,1193.23945,74.887745,1208.366068,1191.385932,3.367612,1,1,1,1,1,0
203,2017-01-31 15:00:00,1212.94,1215.41,1211.77,1212.51,75073,1194.7566,1193.59475,73.745489,1210.654529,1190.619471,3.903585,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37774,2023-06-16 09:00:00,1963.15,1964.71,1963.15,1964.33,4098,1949.2434,1955.61310,66.591302,1963.886858,1952.480142,4.246097,1,0,1,1,1,0
37775,2023-06-16 10:00:00,1964.31,1965.61,1963.06,1964.85,3714,1949.6150,1955.63565,67.177643,1964.632911,1952.938089,4.388206,1,0,1,1,1,0
37776,2023-06-16 11:00:00,1964.91,1966.54,1963.91,1965.62,5336,1949.9344,1955.65960,68.071247,1965.373068,1953.447932,4.510962,1,0,1,1,1,0
37777,2023-06-16 12:00:00,1965.66,1967.93,1962.40,1962.61,7713,1950.1894,1955.66720,61.071618,1965.680284,1953.745716,4.315617,1,0,0,1,1,0
